In [ ]:
root_path = '/content/drive/My Drive/Colab Notebooks/Paper/EXP/'
expr_path = root_path + 'Benchmark/Copy Task/expRNN(Recall 10,Delay 2000)'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import_path = [root_path + subpath for subpath in ['sources/', 'sources/expRNN/']]
for path in import_path:
  sys.path.append(path)

In [ ]:
import sys
import argparse
sys.argv = ['']
parser = argparse.ArgumentParser(description='expRNN Copying Memory Task (Recall 10,Delay 2000)')
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--hidden_size', type=int, default=170)
parser.add_argument('--iterations', type=int, default=4000)
parser.add_argument('--use_adam', type=bool, default=False)
parser.add_argument('--adam_lr', type=float, default=1e-3)
parser.add_argument('--rmsprop_lr', type=float, default=7e-4)
parser.add_argument('--rmsprop_constr_lr', type=float, default=7e-5)
parser.add_argument('--alpha', type=float, default=0.99)
parser.add_argument('--clip_norm', type=float, default=-1)
parser.add_argument("--permute", action="store_true", default = False)
parser.add_argument("-m", "--mode",
                    choices=["exprnn", "dtriv", "cayley", "lstm", "rnn"],
                    default="exprnn",
                    type=str)
parser.add_argument('--K', type=str, default="100", help='The K parameter in the dtriv algorithm. It should be a positive integer or "infty".') #default 100
parser.add_argument("--init",
                    choices=["cayley", "henaff"],
                    default="cayley",
                    type=str)
parser.add_argument('--random-seed', type=int, default=5544,
                    help='random seed')
parser.add_argument('--recall_length', type=int, default=10)
parser.add_argument('--delay_length', type=int, default=2000)
parser.add_argument('--rho_rat_den', type=int, default=10)
parser.add_argument('--forget_bias', type=int, default=1)
parser.add_argument('--emsize', type=int, default=0,
                    help='size of word embeddings')
#Setting
#https://arxiv.org/pdf/1901.08428.pdf
#https://arxiv.org/abs/1905.12080
args = parser.parse_args(['--hidden_size', '190', '--rmsprop_lr', '2e-4', '--rmsprop_constr_lr', '2e-5', '-mexprnn', '--init', 'henaff'])
print(args)

Namespace(K='100', adam_lr=0.001, alpha=0.99, batch_size=128, clip_norm=-1, clip_val=-1, delay_length=2000, emsize=0, forget_bias=1, hidden_size=190, init='henaff', iterations=4000, mode='exprnn', permute=False, random_seed=5544, recall_length=10, rho_rat_den=10, rmsprop_constr_lr=2e-05, rmsprop_lr=0.0002, use_adam=False)


In [ ]:
import torch
import numpy as np
import random
import os
# Fix seed across experiments
# Same seed as that used in "Orthogonal Recurrent Neural Networks with Scaled Cayley Transform"
# https://github.com/SpartinStuff/scoRNN/blob/master/scoRNN_copying.py#L79
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(5544)
np.random.seed(5544)
random.seed(5544)

#Deterministic training
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8" #increase library footprint in GPU memory by approximately 24MiB
#os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":16:8" #(may limit overall performance)"

In [ ]:
from trivializations import cayley_map, expm
from initialization import henaff_init_, cayley_init_
from data_module import CopyMemoryDataModule
from torch.nn import Embedding

hidden_size = args.hidden_size
batch_size  = args.batch_size
iterations      = args.iterations
device      = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
iterations_to_save = 1
batch_first = True
many_to_many = True

if args.init == "cayley":
    init =  cayley_init_
elif args.init == "henaff":
    init = henaff_init_

if args.K != "infty":
    args.K = int(args.K)
if args.mode == "exprnn":
    mode = "static"
    param = expm
elif args.mode == "dtriv":
    # We use 100 as the default to project back to the manifold.
    # This parameter does not really affect the convergence of the algorithms, even for K=1
    mode = ("dynamic", args.K, 100)
    param = expm
elif args.mode == "cayley":
    mode = "static"
    param = cayley_map
else:
    mode = None
    param = None

#Setting up data module
# Load data   
datamodule = CopyMemoryDataModule(args.recall_length, args.delay_length, batch_size)

if args.emsize > 0:
  embed_layer = Embedding(datamodule.input_size, args.emsize) # Token2Embeddings
  input_size = args.emsize
else:
  embed_layer = None
  input_size = datamodule.input_size

In [ ]:
from model_loader import Model
from custom_modules import AMSAF_SVD
from orthogonal import modrelu

#Initialize Model
nonlinearity = modrelu(hidden_size)
model = Model(input_size, hidden_size, datamodule.output_size, nonlinearity, initializer_skew = init,
              mode = mode, param = param, args=args, embed_layer = embed_layer, batch_first=batch_first, many_to_many = many_to_many).to(device)
#Initialize Optimizers
unconstrained_parameters = []
constrained_parameters = []

for name, p in model.named_parameters():
    if any(map(name.__contains__, ['recurrent_kernel'])):
        constrained_parameters.append(p)
    else:
        unconstrained_parameters.append(p)
if args.use_adam:
  adam_optim = torch.optim.Adam(unconstrained_parameters, lr=args.adam_lr)
  rmsprop_optim = torch.optim.RMSprop(constrained_parameters, lr=args.rmsprop_constr_lr)
else:
  rmsprop_optim = torch.optim.RMSprop([
                  {'params': unconstrained_parameters},
                  {'params': constrained_parameters, 'lr': args.rmsprop_constr_lr}
              ], lr=args.rmsprop_lr, alpha = args.alpha)
  adam_optim = None

optim_list = [optim for optim in [rmsprop_optim, adam_optim] if optim]
model.setup_optimizer_list(optim_list)

column_names = ['Iteration', 'Training Loss', 'Training Accuracy', 'Test Loss', 'Test Accuracy']
train_log = model.load_log_df(iterations_to_save, expr_path, column_names)

In [ ]:
def pred(model, datamodule, training = True):
  global device, train_log
  batch_x, batch_y = datamodule.enum()
  batch_x, batch_y = batch_x.to(dtype = torch.float32, device = device), batch_y.to(device)
  state = None
  state = model.state_reprocess(batch_x, state)
  
  logits = model.compute_logits(batch_x, state)
  loss = model.loss(logits, batch_y)
  
  if training:
    model.gradient_descent(loss)
  
  with torch.no_grad():
    train_loss = model.average_value(loss.item(), 1, 6)
    train_acc = model.acc(logits, batch_y)
  
  return train_loss, train_acc

In [ ]:
def save_model(iterations_to_save, experiment_path):
  if (model.step % iterations_to_save == 0):
    if ((model.step // iterations_to_save) % 2) == 1:
      model.save_state(experiment_path+'/last_odd.ckpt')
    else:
      model.save_state(experiment_path+'/last_even.ckpt')

In [ ]:
import pandas as pd
#Training
model.train()
for iteration in range(model.step+1, iterations):
  train_loss, train_acc = pred(model, datamodule, training = True)
  print('Train: [{}/{} ({:.0f}%)]\tLoss: {:.3f}\tAccuracy: {:.2f}%'.format(iteration, iterations, 100. * iteration / iterations, train_loss, train_acc))

  model.eval()
  with torch.no_grad():
      test_loss, test_acc = pred(model, datamodule, training = False)
  if len(train_log) == 0:
    best_test_acc = test_acc
  else:
    best_test_acc = max(train_log["Test Accuracy"].max(), test_acc)
  print('Test Iteration {}| Loss: {:.3f}\tAccuracy: {:.2f}%\tBest Accuracy: {:.2f}'.format(iteration, test_loss, test_acc, best_test_acc))

  model.step = iteration #update_log_df and save_model requires this command
  new_row = pd.Series({"Iteration": iteration, "Training Loss": train_loss, "Training Accuracy": train_acc,
                        "Test Loss": test_loss, 'Test Accuracy': test_acc})
  train_log = model.update_log_df(new_row, train_log, expr_path+'/train_log.pkl')
  save_model(iterations_to_save, expr_path)
  model.train()

Train: [0/4000 (0%)]	Loss: 3.744	Accuracy: 11.79%
Test Iteration 0| Loss: 2.244	Accuracy: 26.30%	Best Accuracy: 26.30
Train: [1/4000 (0%)]	Loss: 2.251	Accuracy: 26.17%
Test Iteration 1| Loss: 1.526	Accuracy: 44.76%	Best Accuracy: 44.76


KeyboardInterrupt: ignored

In [ ]:
model.train_plot(train_log, 'Training Loss', 'Test Loss')

In [ ]:
model.train_plot(train_log, 'Training Accuracy', 'Test Accuracy')